In [1]:
from utils import Ising, conditional, log_unnormalized_p
from plot_utils import *
from tqdm.auto import tqdm

In [ ]:
def gibbs_sampling(model, state, n_samples):
    n = model.dim
    model.state = state
    
    samples = []
    states = np.array([[i, j] for i in range(n) for j in range(n)])
    for t in range(n_samples):
        np.random.shuffle(states)
        for i, j in states:
            p_Xij = conditional(model.state, i, j, model.Js, model.Jst)
            model.state[i][j] = np.random.binomial(1, p_Xij) * 2 - 1  # 0 -> -1, 1 -> 1
        samples.append(model.state.copy())

    return samples


In [ ]:
dim = 100
Js = 0
Jsts = [0.25, 0.5, 1, 10]
output_dir = "plots"
save_snapshot_every = 100
n_samples = 400

filenames = []
if output_dir:
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)

for Jst in tqdm(Jsts):
    samples = []
    model = Ising(dim, Js, Jst)

    model.init_state()
    samples.append(model.state.copy())
    samples.extend(gibbs_sampling(model, model.state, n_samples))

    for i in range(0, len(samples), save_snapshot_every):
        sample = samples[i]
        title = f"Dim={model.dim}, Js={model.Js}, Jst={model.Jst}, Time={i}"
        file_path = f"{output_dir}/{title.replace(', ', '_')}.png"
        visualize_state(sample, title, file_path)
        filenames.append(file_path)

  0%|          | 0/4 [00:00<?, ?it/s]

<Figure size 640x480 with 0 Axes>

In [ ]:
generate_gif(samples, "gibbs_sampling.gif")

<Figure size 640x480 with 0 Axes>

In [6]:
merge_images(filenames, 4, 5, "gibbs_sampling.png")